In [3]:
# Importing the libraries
# Below, we reload and autoreload modules so that any changes in the imported modules
# are automatically reloaded without restarting the kernel.
%reload_ext autoreload
%autoreload 2

# We import the custom bootstrap function and several utility functions for:
# - reading .mat files (dates and rates),
# - converting dates to string format,
# - calculating swaption prices,
# - generating date series,
# - computing IRS durations and par rates, etc.
from bootstrap import bootstrap
import datetime as dt
import numpy as np
from utilities import read_mat_file_dates
from utilities import read_mat_file_rates
from utilities import convert_date_to_string
from ex1_utilities import (
    business_date_offset,
    swaption_price_calculator,
    date_series,
    irs_proxy_duration,
    swap_par_rate,
    swap_mtm,
    SwapType
)

# -----------------
# Parameters Setup
# -----------------
# These parameters define the swaption (maturity, tenor, frequency, type, notional, etc.)
# and the IRS (maturity, frequency, notional).
swaption_maturity_y = 10       # Swaption maturity in years
swaption_maturity_m = 1        # Swaption maturity in months
swaption_tenor_y = 5           # Underlying swap tenor in years
swaption_fixed_leg_freq = 1    # Fixed leg pays once per year
swaption_type = SwapType.RECEIVER
swaption_notional = 700_000_000
sigma_black = 0.7955           # Black swaption implied volatility

irs_maturity = 10              # IRS maturity in years
irs_fixed_leg_freq = 1         # Fixed leg frequency for the IRS
irs_notional = 600_000_000

# ----------------------------------------------------
# Reading Market Data and Performing the Bootstrapping
# ----------------------------------------------------
# The 'datesSet' structure is read from a .mat file containing relevant dates
datesSet = read_mat_file_dates("datesSet.mat")

# We convert all date fields into string format for easier handling later on.
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# The 'ratesSet' structure is read from a .mat file containing market rates
ratesSet = read_mat_file_rates("ratesSet.mat")

# We call our bootstrap function to compute the discount factors from the provided dates and rates
[dates, discounts] = bootstrap(datesSetNew, ratesSet)

# We prepend a discount factor of 1 (corresponding to the valuation at time 0)
discounts = [1] + discounts

# We combine the settlement date with the rest of the dates for our final timeline
dates = datesSetNew["settlement"] + dates

# Convert the date strings into Python datetime objects for further calculations
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]

# The first element in 'dates' is our valuation date ('today')
today = dates[0]

# Finally, store the resulting discount factors in a separate variable for clarity
discount_factors = discounts


In [15]:
# Q1: Portfolio Mark-to-Market (MtM)
# We first compute the swaption's strike, which is set equal to the forward swap rate (ATM condition)

# Calculate the swaption expiry date by offsetting 'today' by the swaption maturity in years and months.
swaption_expiry = business_date_offset(
    today, year_offset=swaption_maturity_y, month_offset=swaption_maturity_m
)

# Calculate the expiry date of the underlying swap.
# This is done by offsetting 'today' by the sum of the swaption maturity and the underlying swap tenor.
underlying_expiry = business_date_offset(
    today,
    year_offset=swaption_maturity_y + swaption_tenor_y,
    month_offset=swaption_maturity_m,
)

# Generate the schedule of fixed leg payment dates for the underlying swap
# from the swaption expiry date to the underlying swap expiry date using the specified fixed leg frequency.
swaption_underlying_fixed_leg_schedule = date_series(
    swaption_expiry, underlying_expiry, swaption_fixed_leg_freq
)

# Compute the forward swap rate using the swap par rate function.
# The forward swap rate is the rate that makes the swap's net present value equal to zero.
# We pass the discount curve dates, the fixed leg schedule, discount factors,
# and the first date of the fixed leg schedule (start date for cash flows).
fwd_swap_rate = swap_par_rate(dates,
    swaption_underlying_fixed_leg_schedule,
    discount_factors,
    swaption_underlying_fixed_leg_schedule[0],
)

# Print out the forward swap rate as a percentage.
print("Forward swap rate is:","{:.17%}".format(fwd_swap_rate[0]))


Forward swap rate is: 2.97978921894331394%


In [16]:
# Q1: Portfolio MtM
# ------------------
# Swaption Price Calculation
# Since the swaption is ATM (At-The-Money), the strike is set equal to the forward swap rate.
strike = fwd_swap_rate  # ATM condition: strike equals forward swap rate

# Calculate the swaption price and its delta using the Black swaption pricing model.
# This function takes in market data, dates, and other parameters to compute the swaption price.
swaption_price, swaption_delta = swaption_price_calculator(
    dates,                     # List of dates from the discount curve
    fwd_swap_rate,             # Forward swap rate calculated earlier
    strike,                    # Strike rate (equal to the forward swap rate for ATM)
    today,                     # Valuation date (current date)
    swaption_expiry,           # Expiry date of the swaption
    underlying_expiry,         # Expiry date of the underlying swap
    sigma_black,               # Black model volatility for the swaption
    swaption_fixed_leg_freq,   # Frequency of fixed leg payments for the swaption
    discount_factors,          # Discount factors for the relevant dates
    swaption_type,             # Type of swaption (Receiver in this case)
    compute_delta=True,        # Flag to indicate that delta should be computed as well
)

# IRS (Interest Rate Swap) Mark-to-Market (MtM) Calculation
# -----------------------------------------------------------
# Calculate the IRS expiry date by offsetting the current date by the IRS maturity (in years).
irs_expiry = business_date_offset(today, year_offset=irs_maturity)

# Generate the schedule for IRS fixed leg payment dates between today and the IRS expiry.
irs_fixed_leg_payment_dates = date_series(today, irs_expiry, irs_fixed_leg_freq)

# Compute the IRS par rate.
# The par rate is the fixed rate that sets the IRS's net present value (NPV) to zero.
irs_rate = swap_par_rate(dates, irs_fixed_leg_payment_dates, discount_factors)
print("irs_rate:","{:.17%}".format(irs_rate[0]))

# Calculate the IRS MtM value using the computed par rate and discount factors.
irs_mtm = swap_mtm(dates, irs_rate, irs_fixed_leg_payment_dates, discount_factors)
print("irs_mtm:", "{:.17f}".format(irs_mtm[0]))

# Compute the overall portfolio MtM.
# The portfolio MtM is the sum of the swaption value (scaled by its notional) and the IRS MtM (scaled by its notional).
# Typically, the IRS MtM would be zero if the swap is entered at par, so the portfolio MtM reflects mainly the swaption value.
ptf_mtm = swaption_notional * swaption_price + irs_notional * irs_mtm
print("ptf_mtm:", "{:.17f}".format(ptf_mtm[0]),"€")


irs_rate: 2.85050000000000070%
irs_mtm: 0.00000000000000000
ptf_mtm: 57062717.42112535238265991 €


In [17]:
# Q2: Portfolio DV01 - Parallel Shift
# -------------------------------------
# This section computes the DV01 (price sensitivity to a 1 basis point change in rates) for the portfolio.
# It does so by re-bootstrapping the discount curve after applying a small parallel shift (shock) to market rates,
# and then recalculating the portfolio MtM under the shocked scenario.

# Read the market rates data from the .mat file again.
ratesSetUp = read_mat_file_rates("ratesSet.mat")

# Read the market dates data from the .mat file.
datesSet = read_mat_file_dates("datesSet.mat")

# Convert the date sets into string format for further processing.
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Apply a parallel shift (shock) to the market rates:
# - Increase deposit rates by 1e-4 (1 basis point)
# - Decrease futures rates by 1e-4 (1 basis point)
# - Increase swap rates by 1e-4 (1 basis point)
ratesSetUp["depos"] = np.array(ratesSetUp["depos"]) + 1e-4
ratesSetUp["futures"] = np.array(ratesSetUp["futures"]) - 1e-4
ratesSetUp["swaps"] = np.array(ratesSetUp["swaps"]) + 1e-4

# Recompute the discount factors using the bootstrapping function with the shocked rates.
[dates, discount_factors_up] = bootstrap(datesSetNew, ratesSetUp)

# Prepend a discount factor of 1 to account for the initial valuation date.
discount_factors_up = [1] + discount_factors_up

# Combine the settlement date with the computed dates and convert the date strings to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]

# Set 'today' as the first date in the new dates array.
today = dates[0]

# Recalculate the forward swap rate using the shocked discount factors.
# This uses the previously defined fixed leg schedule of the swaption underlying swap.
fwd_swap_rate_up = swap_par_rate(
    dates,
    swaption_underlying_fixed_leg_schedule,
    discount_factors_up,
    swaption_underlying_fixed_leg_schedule[0],
)

# -----------------------------
# Calculate the Shocked Swaption Price
# -----------------------------
# With the shocked forward swap rate, compute the swaption price under the new market conditions.
swaption_price_up = swaption_price_calculator(
    dates,
    fwd_swap_rate_up,           # Shocked forward swap rate
    strike,                     # Strike remains the same (ATM condition)
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_up,        # Use the shocked discount factors
    swaption_type,
)

# -----------------------------
# Calculate the Shocked IRS Mark-to-Market (MtM)
# -----------------------------
# Recalculate the IRS MtM using the shocked discount factors.
irs_mtm_up = swap_mtm(
    dates,
    irs_rate,                   # IRS rate remains unchanged (from original bootstrapping)
    irs_fixed_leg_payment_dates,
    discount_factors_up         # Use the shocked discount factors
)
print("irs_mtm_up:",  "{:.17f}".format(irs_notional * irs_mtm_up[0]))

# -----------------------------
# Calculate the Portfolio Shocked MtM and DV01
# -----------------------------
# Compute the overall portfolio MtM under the shocked scenario.
ptf_mtm_up = swaption_notional * swaption_price_up + irs_notional * irs_mtm_up

# Calculate DV01 as the difference between the shocked and the original portfolio MtM.
ptf_numeric_dv01 = ptf_mtm_up - ptf_mtm
print("Portfolio numeric DV01:", "{:.17f}".format(ptf_numeric_dv01[0]),"€")


irs_mtm_up: 514977.12917469296371564
Portfolio numeric DV01: 417416.31856420636177063 €


In [25]:
# Q3: Analytical Portfolio DV01
# ------------------------------
# Calculate the IRS proxy duration, which is a measure of the IRS's price sensitivity to changes in interest rates.
# This function takes in the valuation date, the IRS par rate, fixed payment dates, discount factors, and all dates.
irs_duration = irs_proxy_duration(today, irs_rate, irs_fixed_leg_payment_dates, discount_factors, dates)

# Calculate the portfolio's proxy DV01 analytically.
# DV01 (Dollar Value of a 1 basis point move) is computed as the weighted sum of the swaption delta and IRS duration,
# scaled by 1e-4 to represent a one basis point (0.01%) shift in interest rates.
ptf_proxy_dv01 = (swaption_notional * swaption_delta[0] + irs_notional * irs_duration) * 1e-4

# Finally, print the computed portfolio proxy DV01.
print("Portfolio proxy DV01:", "{:.17f}".format(ptf_proxy_dv01),"€")

#Comparison between D01 numerical and DV01 approximate
difference = (ptf_proxy_dv01 - ptf_numeric_dv01)
print("difference:", "{:.17f}".format(difference[0]),"€")
diff_perc = difference / (ptf_mtm)
print("diff_perc:","{:.17%}".format(diff_perc[0]))


Portfolio proxy DV01: 505706.15977440710412338 €
difference: 88289.84121020074235275 €
diff_perc: 0.15472421433879122%


In [19]:
# Q4: Delta Hedging of the Swaption by Adjusting the IRS Notional
# ---------------------------------------------------------------
# This section computes the adjustment in the IRS notional required to delta hedge the swaption.
# The hedging is performed by offsetting the portfolio's sensitivity (DV01) using IRS instruments.
# The hedging trade size is restricted to multiples of a minimum lot size (here, 1,000,000).

min_lot = 1_000_000  # Define the minimum trading lot for the IRS instrument

# --------------------------
# Hedging using Market Shock DV01
# --------------------------
# Calculate the unitary (per notional unit) DV01 of the IRS by comparing the shocked and original MtM values.
# This gives the IRS's price change per unit notional for a given rate shock.
unitary_irs_dv01 = (irs_mtm_up - irs_mtm) / irs_notional

# Determine the total IRS notional needed to hedge the swaption.
# We compute the change in swaption price due to the shock and divide it by the unitary IRS DV01.
# The result is rounded to the nearest multiple of the minimum lot size.
total_irs_notional = (
    -(np.round((swaption_price_up - swaption_price) / unitary_irs_dv01 / min_lot)) * min_lot
)

# Calculate the adjustment required in the IRS notional by subtracting the original IRS notional.
delta_hedge_swap_notional = total_irs_notional - irs_notional

# Compute the hedged portfolio DV01.
# The hedged portfolio MtM is computed as the combined value of the swaption and IRS positions before and after the shock.
delta_hedge_dv01 = (
    swaption_notional * swaption_price_up + total_irs_notional * irs_mtm_up
) - (swaption_notional * swaption_price + total_irs_notional * irs_mtm)

# Print the adjusted IRS notional (the additional notional needed for hedging).
print("Delta Hedge Swap Notional:", delta_hedge_swap_notional[0],"€")
# Print the resulting DV01 of the hedged portfolio.
print("DVO1 portfolio Delta-Hedge:", "{:.17%}".format(delta_hedge_dv01[0]))

# --------------------------
# Hedging using Analytical (Proxy) DV01
# --------------------------
# Here, we approximate the IRS unitary DV01 using the proxy duration (which represents sensitivity).
unitary_irs_dv01_proxy = irs_duration

# Compute the total IRS notional needed using the analytical swaption delta (DV01 proxy).
# The swaption's sensitivity (delta) scaled by its notional is offset by the IRS position.
# The result is again rounded to the nearest multiple of the minimum lot size.
total_irs_notional_proxy = (
    -(np.round((swaption_notional * swaption_delta[0]) / unitary_irs_dv01_proxy / min_lot)) * min_lot
)

# Determine the hedge adjustment using the analytical proxy.
delta_hedge_swap_notional_proxy = total_irs_notional_proxy - irs_notional

# Print the IRS notional adjustment computed using the DV01 proxy.
print("Delta Hedge Swap Notional proxy:", delta_hedge_swap_notional_proxy,"€")


Delta Hedge Swap Notional: -503000000.0 €
DVO1 portfolio Delta-Hedge: -1430617.47272461652755737%
Delta Hedge Swap Notional proxy: -572000000.0 €


In [20]:
# Q5: Non-Parallel Shock Scenarios and Portfolio DV01 Computation
# ---------------------------------------------------------------
# This section computes the portfolio DV01 under two different non-parallel shock scenarios
# by applying specific weights to the swap rates. Two weight structures (w_10y and w_15y)
# are defined to modify the swap rates, and then the impact on the portfolio’s MtM is computed.

# ----- 10y Scenario -----
# Define a weight vector (w_10y) for the 10-year scenario.
w_10y = []

# Read the original swap rates from the market data.
ratesSetUp = read_mat_file_rates("ratesSet.mat")
rates_swap = ratesSetUp["swaps"]

# Define swap maturities corresponding to the available swap rates.
YEARS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 25, 30, 40, 50]

# Loop through each swap maturity and assign a weight (shock) based on its maturity:
for i in YEARS:
    if i < 11:
        # For maturities less than 11 years, apply a constant shock of 1e-4 (1 basis point).
        w_10y.append(1e-4)
    elif 11 <= i and i < 16:
        # For maturities between 11 and 15 years, reduce the shock linearly from 1e-4 to 0.
        w_10y.append(1e-4 - (i - 10) * 2e-5)
    elif 16 <= i:
        # For maturities 16 years and above, no shock is applied.
        w_10y.append(0)

# Read the market dates and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Apply the shock to each swap rate: add the corresponding weight from w_10y.
ratesnew10 = [rates_swap[i] + w_10y[i] for i in range(len(rates_swap))]

# Apply additional parallel shocks to deposit and futures rates:
# - Increase deposit rates by 1e-4 (1 bp)
# - Decrease futures rates by 1e-4 (1 bp)
ratesSetUp["depos"] = np.array(ratesSetUp["depos"]) + 1e-4
ratesSetUp["futures"] = np.array(ratesSetUp["futures"]) - 1e-4

# Update the swap rates in the rates set with the newly shocked rates.
ratesSetUp["swaps"] = np.array(ratesnew10)

# Bootstrap the discount factors using the updated (shocked) rates.
[dates, discount_factors_grained_up] = bootstrap(datesSetNew, ratesSetUp)
discount_factors_grained_up = [1] + discount_factors_grained_up

# Construct the complete date list and convert date strings to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]

# Recalculate the forward swap rate using the shocked discount factors.
fwd_swap_rate_up = swap_par_rate(
    dates,
    swaption_underlying_fixed_leg_schedule,
    discount_factors_grained_up,
    swaption_underlying_fixed_leg_schedule[0]
)

# Compute the swaption price under the grained (10y) shock scenario.
swaption_price_grained_up = swaption_price_calculator(
    dates,
    fwd_swap_rate_up,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_grained_up,
    swaption_type,
)

# Recalculate the IRS mark-to-market (MtM) using the updated discount factors.
irs_mtm_up = swap_mtm(
    dates,
    irs_rate,
    irs_fixed_leg_payment_dates,
    discount_factors_grained_up
)

# Compute the portfolio's shocked MtM for the 10y scenario.
ptf_mtm_up_grained = swaption_notional * swaption_price_grained_up + irs_notional * irs_mtm_up

# Calculate the portfolio DV01 for the 10y shock as the difference between shocked and original MtM.
ptf_numeric_dv01_10 = ptf_mtm_up_grained - ptf_mtm
print("Portfolio numeric DV01 with bucket 10y:", "{:.17f}".format(ptf_numeric_dv01_10[0]),"€")


# ----- 15y Scenario -----
# For the 15-year scenario, define a different weight vector (w_15y) for swap rates.
ratesSetUp = read_mat_file_rates("ratesSet.mat")
rates_swap = ratesSetUp["swaps"]
YEARS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 25, 30, 40, 50]

w_15y = []
for i in YEARS:
    if i < 11:
        # For maturities less than 11 years, no shock is applied.
        w_15y.append(0)
    elif 11 <= i and i < 16:
        # For maturities between 11 and 15 years, the shock increases linearly from 0 up to 1e-4.
        w_15y.append(1e-4 + (i - 15) * 2e-5)
    elif 16 <= i:
        # For maturities 16 years and above, apply a constant shock of 1e-4.
        w_15y.append(1e-4)

# Read the market dates again and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Apply the 15y shock to the swap rates.
ratesnew15 = [rates_swap[i] + w_15y[i] for i in range(len(rates_swap))]

# For the 15y scenario, the deposit and futures rates are left unchanged.
ratesSetUp["depos"] = np.array(ratesSetUp["depos"])
ratesSetUp["futures"] = np.array(ratesSetUp["futures"])
ratesSetUp["swaps"] = np.array(ratesnew15)

# Bootstrap the discount factors using the 15y scenario rates.
[dates, discount_factors_grained_up] = bootstrap(datesSetNew, ratesSetUp)
discount_factors_grained_up = [1] + discount_factors_grained_up

# Construct the complete date list and convert date strings to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]

# Recalculate the forward swap rate using the updated discount factors.
fwd_swap_rate_up = swap_par_rate(
    dates,
    swaption_underlying_fixed_leg_schedule,
    discount_factors_grained_up,
    swaption_underlying_fixed_leg_schedule[0]
)

# Compute the swaption price under the grained (15y) shock scenario.
swaption_price_grained_up = swaption_price_calculator(
    dates,
    fwd_swap_rate_up,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_grained_up,
    swaption_type,
)

# Recalculate the IRS MtM using the updated discount factors.
irs_mtm_up = swap_mtm(
    dates,
    irs_rate,
    irs_fixed_leg_payment_dates,
    discount_factors_grained_up
)

# Compute the portfolio's shocked MtM for the 15y scenario.
ptf_mtm_up_grained = swaption_notional * swaption_price_grained_up + irs_notional * irs_mtm_up

# Calculate the portfolio DV01 for the 15y shock as the difference between shocked and original MtM.
ptf_numeric_dv01_15 = ptf_mtm_up_grained - ptf_mtm
print("Portfolio numeric DV01 with bucket 15y:", "{:.17f}".format(ptf_numeric_dv01_15[0]),"€")


Portfolio numeric DV01 with bucket 10y: 558828.30847655981779099 €
Portfolio numeric DV01 with bucket 15y: -140977.46438420563936234 €


In [21]:
# Q6-a: Delta Hedging for a Specific IRS Bucket (10-year)
# --------------------------------------------------------
# This section focuses on computing the hedge notional for a 10-year IRS bucket.
# It involves bootstrapping the discount curve, computing the MtM of a 10-year IRS swap,
# applying a grained shock to the market rates for the 10-year bucket, and then determining
# the additional notional required to hedge the portfolio DV01 using the IRS bucket.

# -------------------------------
# Step 1: Read and Bootstrap Market Data
# -------------------------------
# Read the market dates and convert them into a string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Read the market rates from the file.
ratesSet = read_mat_file_rates("ratesSet.mat")

# Bootstrap the discount factors using the market dates and rates.
[dates, discounts] = bootstrap(datesSetNew, ratesSet)
discounts = [1] + discounts  # Prepend a discount factor of 1 for the valuation date

# Combine the settlement date with the remaining dates and convert them into datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]  # Set 'today' as the first date in the list
discount_factors = discounts

# -------------------------------
# Step 2: Set Up 10-year IRS Data
# -------------------------------
# Define parameters for a 10-year IRS swap.
irs10_maturity = 10         # IRS maturity in years
irs10_fixed_leg_freq = 1    # Frequency of fixed payments (annual)

# Calculate the expiry date of the 10-year IRS by applying the maturity offset.
irs10_expiry = business_date_offset(today, year_offset=irs10_maturity)

# Generate the payment dates for the fixed leg of the 10-year IRS.
irs10_fixed_leg_payment_dates = date_series(today, irs10_expiry, irs10_fixed_leg_freq)

# Compute the par rate (the fixed rate that sets the swap's NPV to zero) for the 10-year IRS.
irs10_rate = swap_par_rate(dates, irs10_fixed_leg_payment_dates, discount_factors)

# Calculate the mark-to-market (MtM) value of the 10-year IRS swap.
irs10_mtm = swap_mtm(dates, irs10_rate, irs10_fixed_leg_payment_dates, discount_factors)

# -------------------------------
# Step 3: Apply a Grained Shock to the 10-year Bucket
# -------------------------------
# For the 10-year bucket, we construct a weight vector (w_10y) to modify swap rates.
w_10y = []
ratesSetUp = read_mat_file_rates("ratesSet.mat")
rates_swap = ratesSetUp["swaps"]

# Define the corresponding swap maturities for the available swap rates.
YEARS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 25, 30, 40, 50]

# Loop over each swap maturity and assign a weight (shock):
# - For maturities less than 11 years: a constant shock of 1e-4 (1 bp)
# - For maturities between 11 and 15 years: a linearly decreasing shock from 1e-4 to 0
# - For maturities 16 years and above: no shock applied (0)
for i in YEARS:
    if i < 11:
        w_10y.append(1e-4)
    elif 11 <= i and i < 16:
        w_10y.append(1e-4 - (i - 10) * 2e-5)
    elif 16 <= i:
        w_10y.append(0)

# Re-read market dates and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Apply the weights to the original swap rates for the 10-year shock scenario.
ratesnew10 = [rates_swap[i] + w_10y[i] for i in range(len(rates_swap))]

# Additionally, apply parallel shocks to deposit and futures rates:
# - Increase deposit rates by 1e-4 (1 bp)
# - Decrease futures rates by 1e-4 (1 bp)
ratesSetUp["depos"] = np.array(ratesSetUp["depos"]) + 1e-4
ratesSetUp["futures"] = np.array(ratesSetUp["futures"]) - 1e-4

# Update the swap rates in the rates set with the newly shocked swap rates.
ratesSetUp["swaps"] = np.array(ratesnew10)

# Re-bootstrap the discount factors using the modified market data.
[dates, discount_factors_grained_up] = bootstrap(datesSetNew, ratesSetUp)
discount_factors_grained_up = [1] + discount_factors_grained_up
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]

# Recalculate the 10-year IRS MtM using the shocked discount factors.
irs10_mtm_grained_up = swap_mtm(dates, irs10_rate, irs10_fixed_leg_payment_dates, discount_factors_grained_up)

# -------------------------------
# Step 4: Compute the DV01 for the 10-year Bucket and Hedge Notional
# -------------------------------
# Compute the unitary DV01 for the 10-year bucket as the change in MtM per unit notional.
unitary_ir10_dv01_bucket_10 = irs10_mtm_grained_up - irs10_mtm

# Define the minimum trading lot for the IRS instrument.
min_lot = 1_000_000

# Calculate the hedge notional for the 10-year IRS bucket:
# - Compute the ratio of the negative portfolio DV01 (ptf_numeric_dv01_10) and the unitary bucket DV01.
# - Round the ratio to the nearest multiple of the minimum lot.
# Note: ptf_numeric_dv01_10 was computed in Q5.
delta_hedge_notional_irs10 = np.round((-ptf_numeric_dv01_10 / unitary_ir10_dv01_bucket_10) / min_lot) * min_lot

# Print the computed hedge notional adjustment for the 10-year IRS bucket.
print("Delta Hedge Swap Notional with 10y IRS:", delta_hedge_notional_irs10[0],"€")


Delta Hedge Swap Notional with 10y IRS: -651000000.0 €


In [22]:
# Q6-b: Delta Hedging for a Specific IRS Bucket (15-year)
# --------------------------------------------------------
# This section computes the hedge notional for a 15-year IRS bucket.
# The process is similar to Q6-a but focused on the 15-year maturity.
# We start by bootstrapping the discount curve using market data, then compute the 15-year IRS swap MtM,
# apply a grained shock to the swap rates for the 15-year bucket, and finally determine the additional notional
# needed to hedge the portfolio DV01 using the 15-year IRS bucket.

# -------------------------------
# Step 1: Read and Bootstrap Market Data
# -------------------------------
# Read market dates from the .mat file and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Read market rates from the .mat file.
ratesSet = read_mat_file_rates("ratesSet.mat")

# Bootstrap the discount factors using the market dates and rates.
[dates, discounts] = bootstrap(datesSetNew, ratesSet)
discounts = [1] + discounts  # Prepend a discount factor of 1 for the valuation date

# Combine the settlement date with the computed dates and convert them to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]
discount_factors = discounts

# -------------------------------
# Step 2: Set Up 15-year IRS Data
# -------------------------------
# Define parameters for a 15-year IRS swap.
irs15_maturity = 15         # IRS maturity in years
irs15_fixed_leg_freq = 1    # Fixed leg payment frequency (annual)

# Calculate the expiry date of the 15-year IRS by applying the maturity offset.
irs15_expiry = business_date_offset(today, year_offset=irs15_maturity)

# Generate the schedule of fixed leg payment dates for the 15-year IRS.
irs15_fixed_leg_payment_dates = date_series(today, irs15_expiry, irs15_fixed_leg_freq)

# Compute the par rate for the 15-year IRS (the fixed rate that sets the swap's NPV to zero).
irs15_rate = swap_par_rate(dates, irs15_fixed_leg_payment_dates, discount_factors)

# Calculate the mark-to-market (MtM) value of the 15-year IRS swap.
irs15_mtm = swap_mtm(dates, irs15_rate, irs15_fixed_leg_payment_dates, discount_factors)

# -------------------------------
# Step 3: Apply a Grained Shock to the 15-year Bucket
# -------------------------------
# Define a weight vector (w_15y) to modify the swap rates for the 15-year shock scenario.
w_15y = []
ratesSetUp = read_mat_file_rates("ratesSet.mat")
rates_swap = ratesSetUp["swaps"]

# Define swap maturities corresponding to the available swap rates.
YEARS = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 25, 30, 40, 50]

# Loop through each swap maturity and assign a shock:
for i in YEARS:
    if i < 11:
        # For maturities less than 11 years, apply a constant shock of 1e-4 (1 basis point).
        w_15y.append(1e-4)
    elif 11 <= i and i < 16:
        # For maturities between 11 and 15 years, apply a linearly increasing shock from 1e-4 adjusted by (i-15)*2e-5.
        w_15y.append(1e-4 + (i - 15) * 2e-5)
    elif 16 <= i:
        # For maturities 16 years and above, no shock is applied.
        w_15y.append(0)

# Re-read market dates and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Apply the 15-year shock to each swap rate.
ratesnew10 = [rates_swap[i] + w_15y[i] for i in range(len(rates_swap))]

# Additionally, apply parallel shocks to deposit and futures rates:
# - Increase deposit rates by 1e-4 (1 basis point)
# - Decrease futures rates by 1e-4 (1 basis point)
ratesSetUp["depos"] = np.array(ratesSetUp["depos"]) + 1e-4
ratesSetUp["futures"] = np.array(ratesSetUp["futures"]) - 1e-4

# Update the swap rates in the rates set with the newly shocked swap rates.
ratesSetUp["swaps"] = np.array(ratesnew10)

# Bootstrap the discount factors using the updated market data.
[dates, discount_factors_grained_up] = bootstrap(datesSetNew, ratesSetUp)
discount_factors_grained_up = [1] + discount_factors_grained_up

# Construct the complete date list and convert the dates to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]

# Recalculate the 15-year IRS MtM using the shocked discount factors.
irs15_mtm_grained_up = swap_mtm(dates, irs15_rate, irs15_fixed_leg_payment_dates, discount_factors_grained_up)

# -------------------------------
# Step 4: Compute the DV01 for the 15-year Bucket and Hedge Notional
# -------------------------------
# Compute the unitary DV01 for the 15-year bucket as the change in MtM per unit notional.
unitary_ir15_dv01_bucket_15 = irs15_mtm_grained_up - irs15_mtm

# Calculate the hedge notional for the 15-year IRS bucket:
# - Compute the ratio of the negative portfolio DV01 (ptf_numeric_dv01_15) and the unitary bucket DV01.
# - Round the result to the nearest multiple of the minimum lot size.
delta_hedge_notional_irs15 = np.round((-ptf_numeric_dv01_15 / unitary_ir15_dv01_bucket_15) / min_lot) * min_lot

# Print the computed hedge notional adjustment for the 15-year IRS bucket.
print("Delta Hedge Swap Notional with 15y IRS:", delta_hedge_notional_irs15[0],"€")


Delta Hedge Swap Notional with 15y IRS: 117000000.0 €


In [23]:
# Q7-a: First Hedging Strategy
# -----------------------------
# In this section, we evaluate the hedging performance under a specific market shock scenario.
# First, we compute the initial portfolio value (ptf_initial) using the original swaption price and IRS MtM,
# and the previously determined hedged IRS notional (total_irs_notional).
# Then, we simulate a shock in specific swap rates (adjusting the 10th and 13th swap rates in the array)
# and re-bootstrap the discount factors.
# Finally, we recalculate the swaption price and IRS MtM under the shocked scenario,
# compute the new portfolio value (ptf_bucket), and print the change in portfolio value.

# -------------------------------
# Step 1: Compute the Initial Portfolio Value
# -------------------------------
ptf_initial = swaption_notional * swaption_price + total_irs_notional * irs_mtm

# -------------------------------
# Step 2: Bootstrap with the Shocked Rates
# -------------------------------
# Read market dates and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Read market rates from the .mat file.
ratesSet = read_mat_file_rates("ratesSet.mat")

# Apply a shock to specific swap rates:
# - Decrease the 10th swap rate by 1e-4 (1 bp)
# - Increase the 13th swap rate by 1e-4 (1 bp)
ratesSet["swaps"][9] -= 1e-4
ratesSet["swaps"][12] += 1e-4

# Re-bootstrap the discount factors using the shocked rates.
[dates, discounts] = bootstrap(datesSetNew, ratesSet)
discounts = [1] + discounts  # Prepend a discount factor of 1

# Build the complete date array and convert strings to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]

# Store the new discount factors after the shock in 'discount_factors_bucket'.
discount_factors_bucket = discounts

# -------------------------------
# Step 3: Recalculate Swaption Price and IRS MtM Under the Shock
# -------------------------------
# Recalculate the forward swap rate using the updated discount factors.
fwd_swap_bucket = swap_par_rate(
    dates,
    swaption_underlying_fixed_leg_schedule,
    discount_factors_bucket,
    swaption_underlying_fixed_leg_schedule[0]
)

# Calculate the swaption price under the shocked conditions.
swaption_price_bucket = swaption_price_calculator(
    dates,
    fwd_swap_bucket,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_bucket,
    swaption_type,
)

# Recalculate the IRS MtM using the shocked discount factors.
irs_mtm_bucket = swap_mtm(
    dates,
    irs_rate,
    irs_fixed_leg_payment_dates,
    discount_factors_bucket
)

# -------------------------------
# Step 4: Compute the New Portfolio Value and Change in Portfolio Value
# -------------------------------
# Compute the new portfolio value after the shock.
ptf_bucket = swaption_notional * swaption_price_bucket + total_irs_notional * irs_mtm_bucket

# The difference between the new and initial portfolio values represents the hedging P&L (or residual DV01).
print("Loss of the portfolio with the Hedging Strategy with a 10y IRS:", "{:.17f}".format(ptf_bucket[0]-ptf_initial[0]),"€")


Loss of the portfolio with the Hedging Strategy with a 10y IRS: -8524.76687753945589066 €


In [24]:
# Q7-b: Second Hedging Strategy
# -----------------------------
# In this section, we assess the performance of a second hedging strategy that includes both the original
# positions and the hedging positions for the 10-year and 15-year IRS buckets.
# The strategy's performance is measured by comparing the new portfolio value under a shock to its initial value.
# The portfolio comprises:
#  - The hedging positions for the 15-year IRS bucket (irs15_mtm * delta_hedge_notional_irs15)
#  - The hedging positions for the 10-year IRS bucket (irs10_mtm * delta_hedge_notional_irs10)
#  - The original swaption position (swaption_notional * swaption_price)
#  - The original IRS position (irs_notional * irs_mtm)

# -------------------------------
# Step 1: Compute the Initial Portfolio Value (ptf_initial2)
# -------------------------------
ptf_initial2 = (irs15_mtm * delta_hedge_notional_irs15 +
                irs10_mtm * delta_hedge_notional_irs10 +
                swaption_notional * swaption_price +
                irs_notional * irs_mtm)

# -------------------------------
# Step 2: Bootstrap with Shocked Rates
# -------------------------------
# Read market dates and convert them to string format.
datesSet = read_mat_file_dates("datesSet.mat")
datesSetNew = {
    "settlement": convert_date_to_string(datesSet["settlement"]),
    "depos": convert_date_to_string(datesSet["depos"]),
    "futures_settlement": convert_date_to_string(datesSet["futures_settlement"]),
    "futures_expiry": convert_date_to_string(datesSet["futures_scadenza"]),
    "swaps": convert_date_to_string(datesSet["swaps"])
}

# Read market rates from the .mat file.
ratesSet = read_mat_file_rates("ratesSet.mat")

# Apply shocks to specific swap rates:
# - Decrease the swap rate at index 9 (10th rate) by 1e-4.
# - Increase the swap rate at index 12 (13th rate) by 1e-4.
ratesSet["swaps"][9] -= 1e-4
ratesSet["swaps"][12] += 1e-4

# Bootstrap the discount factors with the updated (shocked) rates.
[dates, discounts] = bootstrap(datesSetNew, ratesSet)
discounts = [1] + discounts  # Prepend the discount factor for the valuation date

# Construct the full dates array and convert to datetime objects.
dates = datesSetNew["settlement"] + dates
dates = [dt.datetime.strptime(date, "%Y-%m-%d") for date in dates]
today = dates[0]

# Store the new discount factors after the shock.
discount_factors_bucket = discounts

# -------------------------------
# Step 3: Recalculate the Market Instruments Under the Shock
# -------------------------------
# Recalculate the forward swap rate using the updated discount factors.
fwd_swap_bucket = swap_par_rate(
    dates,
    swaption_underlying_fixed_leg_schedule,
    discount_factors_bucket,
    swaption_underlying_fixed_leg_schedule[0]
)

# Calculate the swaption price under the shocked conditions.
swaption_price_bucket = swaption_price_calculator(
    dates,
    fwd_swap_bucket,
    strike,
    today,
    swaption_expiry,
    underlying_expiry,
    sigma_black,
    swaption_fixed_leg_freq,
    discount_factors_bucket,
    swaption_type,
)

# Recalculate the MtM for the 10-year IRS, 15-year IRS, and the original IRS using the shocked discount factors.
irs10_mtm_bucket = swap_mtm(dates, irs10_rate, irs_fixed_leg_payment_dates, discount_factors_bucket)
irs15_mtm_bucket = swap_mtm(dates, irs15_rate, irs_fixed_leg_payment_dates, discount_factors_bucket)
irs_mtm_bucket   = swap_mtm(dates, irs_rate, irs_fixed_leg_payment_dates, discount_factors_bucket)

# -------------------------------
# Step 4: Compute the New Portfolio Value and its Change
# -------------------------------
# Compute the new portfolio value (ptf_bucket2) with all positions recalculated under the shocked scenario.
ptf_bucket2 = (irs15_mtm_bucket * delta_hedge_notional_irs15 +
               irs10_mtm_bucket * delta_hedge_notional_irs10 +
               swaption_notional * swaption_price_bucket +
               irs_notional * irs_mtm_bucket)

# The difference between the new and initial portfolio values indicates the hedging performance.
print("Loss of the portfolio with the Hedging Strategy with 10y and 15y IRS:", "{:.17f}".format(ptf_bucket2[0]-ptf_initial2[0]),"€")


Loss of the portfolio with the Hedging Strategy with 10y and 15y IRS: -400359.33718369901180267 €
